In [ ]:
# from src.utils.functions import (
#     doc_conv, check_ocr, free_memory, format_msgs, pdf_loader,
#     modify_orig, delete_papers, get_responses , post_request_with_retries, prompt_prep,
#     resize_image, data_preparation, models_local, models_used,get_img_summary, qdrant_process,
#     api_models_one_img, save_to_pickle, upsert_to_qdrant,colpali_qdrant,
#     encode_image, show_results, convert_pdfs_to_images,
#     check_vllm_status, monitor_vllm_process, retrieve_colpali,process_models,
#     update_vd_new_user,make_tarfile, extract_tarfile, new_user_set_files
#     #CustomRetriever,summarise_context,api_models_imgs

# )


import docling
from transformers import AutoTokenizer
from langchain_huggingface import HuggingFaceEmbeddings
import os

import torch
torch.classes.__path__ = []
import onnxruntime as rt

from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore


In [ ]:
OPEN_MODEL = "google/gemma-3-27b-it"
OPEN_MODEL_SHORT = "Gemma3-27b"

LI_MODEL = "vidore/colpali-v1.3-merged"
LI_MODEL_SHORT = "ColPali"

EMBED_MODEL_ID = "BAAI/bge-base-en-v1.5"
MAX_TOKENS = 512
EMB_DIM = 768

embeddings = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL_ID,
    model_kwargs={"device": "cuda"}
)
emb_tokenizer = AutoTokenizer.from_pretrained(EMBED_MODEL_ID )

PAPERS_DIR ="./papers/"
#VD_DIR ="./src/vectordb/"
QDRANT_PORT= 6333


In [ ]:
from huggingface_hub import login, snapshot_download
login(token=os.environ["HUGGING_FACE_HUB_TOKEN"])

snapshot_download(repo_id=OPEN_MODEL ,cache_dir = os.environ["HF_DIR"] + "hub/")#, force_download=True   )

In [ ]:
MODELS = [
    {"model_name": "gpt-4o-2024-11-20", "model_short": "gpt-4o", "port": "9999", "text_vd": "RAG_TEXT", "mm_vd": "MM_04_GPT_4o", "late_inter": LI_MODEL, "late_inter_short": LI_MODEL_SHORT }, 
    {"model_name": "gpt-4o-mini-2024-07-18", "model_short": "gpt-4o-mini", "port": "9999", "text_vd": "RAG_TEXT", "mm_vd": "MM_05_GPT_4o_mini", "late_inter": LI_MODEL, "late_inter_short": LI_MODEL_SHORT}, 
    {"model_name": OPEN_MODEL, "model_short": OPEN_MODEL_SHORT, "port": "8006", "text_vd": "RAG_TEXT", "mm_vd": "MM_07_GEMMA3_27B", "late_inter": LI_MODEL, "late_inter_short": LI_MODEL_SHORT}]

In [ ]:
papers = [os.path.join(PAPERS_DIR, f) for f in sorted(os.listdir(PAPERS_DIR)) if f.lower().endswith('.pdf')]

In [ ]:
#add doi manually!!! or empty list same length as papers
# doi_papers = [
# "https://doi.org/10.1038/s41590-024-01916-8",
# "https://doi.org/10.1186/s12967-018-1695-0",
# "https://doi.org/10.1097/hjh.0000000000002963",
# ...
# ]
doi_papers = ["" for _ in papers]

In [ ]:
processed_multi, processed_text = pdf_loader (papers, doi_papers, [paper.split("/")[-1] for paper in papers],  os.environ["VD_DIR"], emb_tokenizer)

In [ ]:
#placeholder- modify as needed
prompt = 'You are an expert in glycan biology and you will be querried. Here is the query: {query}\nTask:\nAnswer the clearly and concisely. You will be given Context information, which can be empty.\n Tone: scientific and concise. It should include critical numeric data, significant results, and relevant keywords if relevant.\nConstraints:\nAvoid generic answerd.\nHere is the Context information: \n'

In [ ]:
import asyncio 

dict_outputs = await process_models (processed_multi, prompt, MODELS)
dict_outputs["text_only"]= processed_text

print(dict_outputs.keys())

In [ ]:
#colpali and similar model/processor retriever
# colpali modules
#poetry add git+https://github.com/illuin-tech/colpali.git
#from colpali_engine.models import ColIdefics3, ColIdefics3Processor
#from colpali_engine.models import ColQwen2_5Omni, ColQwen2_5OmniProcessor
#from colpali_engine.models import ColQwen2_5, ColQwen2_5_Processor

#poetry add git+https://github.com/AhmedMasryKU/colflor.git
from colpali_engine.models import ColPali, ColPaliProcessor
from colpali_engine.models import ColFlor, ColFlorProcessor

from transformers import AutoProcessor, AutoModel
from transformers.utils.import_utils import is_flash_attn_2_available

def get_ret_model_processor (MODEL_RET, device):
    
    if MODEL_RET == "vidore/colpali-v1.3-merged":
        processor = ColPaliProcessor.from_pretrained(MODEL_RET)

        model = ColPali.from_pretrained(
            MODEL_RET,
            torch_dtype=torch.bfloat16,
            device_map=device,
            attn_implementation="flash_attention_2" if is_flash_attn_2_available() else None,
        ).eval()
    elif MODEL_RET == "ahmed-masry/ColFlor":
        model = ColFlor.from_pretrained(
            MODEL_RET,
            device_map=device,
            attn_implementation="flash_attention_2" if is_flash_attn_2_available() else None,
        ).eval()

        processor = ColFlorProcessor.from_pretrained(MODEL_RET)
    
    elif MODEL_RET == "vidore/colSmol-500M":
        model = ColIdefics3.from_pretrained(
            MODEL_RET,
            device_map=device,
            attn_implementation="flash_attention_2" if is_flash_attn_2_available() else None,
        ).eval()

        processor = ColIdefics3Processor.from_pretrained(MODEL_RET)

    elif MODEL_RET == "ibm-granite/granite-vision-3.3-2b-embedding":
        model = AutoModel.from_pretrained(
            MODEL_RET,
            device_map=device,
            attn_implementation="flash_attention_2" if is_flash_attn_2_available() else None,
        ).eval()

        processor = AutoProcessor.from_pretrained(MODEL_RET)
        
    elif MODEL_RET == "vidore/colqwen2.5-v0.2":
        model = ColQwen2_5.from_pretrained(
            MODEL_RET,
            torch_dtype=torch.bfloat16,
            device_map=device,  # or "mps" if on Apple Silicon
            attn_implementation="flash_attention_2" if is_flash_attn_2_available() else None,
            ).eval()
        processor = ColQwen2_5_Processor.from_pretrained(MODEL_RET)
    else:
        print(f"Select correct MODEL_RET, current {MODEL_RET} not correct")
        return -1
    return model, processor

In [ ]:
from qdrant_client.http import models

#qdrant should be running in docker !
qdrant_client = QdrantClient(url=f"http://localhost:{QDRANT_PORT}", api_key=os.environ["QDRANT_API_KEY"])

id_text_rag = 0
for idx, model in enumerate(MODELS):
    if idx == 0 and id_text_rag == 0:
        collection_name = model["text_vd"]
        if not qdrant_client.collection_exists(collection_name=collection_name):
            qdrant_client.create_collection(
                collection_name=collection_name,
                on_disk_payload=True,
                vectors_config=models.VectorParams(size=EMB_DIM, on_disk=True, distance=models.Distance.COSINE),
            )
        QdrantVectorStore.from_documents(
            documents=dict_outputs[model["model_short"]],
            url=f"http://localhost:{QDRANT_PORT}",
            api_key=os.environ["QDRANT_API_KEY"],
            collection_name=collection_name,
            embedding=embeddings,
            )
        id_text_rag = 1        

    if not qdrant_client.collection_exists(collection_name=model["mm_vd"]):
        qdrant_client.create_collection(
                collection_name=model["mm_vd"],
                on_disk_payload=True,
                vectors_config= models.VectorParams(
                    size=EMB_DIM,
                    on_disk=True, 
                    distance=models.Distance.COSINE
                    ),
        )
    QdrantVectorStore.from_documents(
    documents=dict_outputs[model["model_short"]],
    url=f"http://localhost:{QDRANT_PORT}",
    api_key=os.environ["QDRANT_API_KEY"],
    collection_name=model["mm_vd"],
    embedding=embeddings,
    )

    if not qdrant_client.collection_exists(collection_name=model["late_inter_short"]):
        qdrant_client.create_collection(
            collection_name=model["late_inter_short"],
            on_disk_payload=True,
            vectors_config=models.VectorParams(
                size=vector_size,
                distance=models.Distance.COSINE,
                on_disk=True,
                multivector_config=models.MultiVectorConfig(
                    comparator=models.MultiVectorComparator.MAX_SIM
                ),
            ),
        )
    model_li, processor_li = get_ret_model_processor (model["late_inter"], "cuda")
    points = create_document_embeddings(PAPERS_DIR, model_li, processor_li batch_size=4)
    qdrant_client.upsert(collection_name=model["late_inter_short"], points=points)
    all_images = convert_pdf_to_images(PAPERS_DIR)
